In [379]:
import os
import cv2
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False
from func import *
import json

### 1. annotation 가져오기

In [380]:
# JSON 파일 경로
json_file_path = 'person_keypoints_val2017.json'

# JSON 파일 읽기
with open(json_file_path, 'r') as f:
    data = json.load(f)

In [381]:
data.keys()

dict_keys(['info', 'licenses', 'images', 'annotations', 'categories'])

In [382]:
data['annotations'][0].keys()

dict_keys(['segmentation', 'num_keypoints', 'area', 'iscrowd', 'keypoints', 'image_id', 'bbox', 'category_id', 'id'])

In [383]:
image_id = []
bbox = []

for item in data['annotations']:
    image_id.append(item['image_id'])
    bbox.append(item['bbox'])

In [384]:
gt_df = pd.DataFrame({'image_id':image_id, 'bbox':bbox})
gt_df

,image_id,bbox
0,425226,"[73.35, 206.02, 300.58, 372.5]"
1,440475,"[491.91, 183.51, 34.85, 73.4]"
2,266400,"[36.32, 0.59, 38.56, 53.86]"
3,69213,"[276.35, 9.6, 244.69, 400.13]"
4,555050,"[97.1, 185.73, 28.66, 27.72]"
...,...,...
10999,259690,"[27, 98, 225, 114]"
11000,145020,"[27, 201, 209, 68]"
11001,562818,"[353, 229, 180, 39]"
11002,439994,"[0, 0, 427, 458]"


In [385]:
gt_df["width"] = gt_df["bbox"].apply(lambda bbox: bbox[2])
gt_df["height"] = gt_df["bbox"].apply(lambda bbox: bbox[3])
gt_df["size"] = gt_df["width"]*gt_df["height"]

In [386]:
gt_df

,image_id,bbox,width,height,size
0,425226,"[73.35, 206.02, 300.58, 372.5]",300.58,372.50,111966.0500
1,440475,"[491.91, 183.51, 34.85, 73.4]",34.85,73.40,2557.9900
2,266400,"[36.32, 0.59, 38.56, 53.86]",38.56,53.86,2076.8416
3,69213,"[276.35, 9.6, 244.69, 400.13]",244.69,400.13,97907.8097
4,555050,"[97.1, 185.73, 28.66, 27.72]",28.66,27.72,794.4552
...,...,...,...,...,...
10999,259690,"[27, 98, 225, 114]",225.00,114.00,25650.0000
11000,145020,"[27, 201, 209, 68]",209.00,68.00,14212.0000
11001,562818,"[353, 229, 180, 39]",180.00,39.00,7020.0000
11002,439994,"[0, 0, 427, 458]",427.00,458.00,195566.0000


### 2. model result에서 score 가져오기

In [387]:
# JSON 파일 경로
json_file_path = 'result_keypoints.json'

# JSON 파일 읽기
with open(json_file_path, 'r') as f:
    result = json.load(f)

In [388]:
result[0].keys()

dict_keys(['category_id', 'center', 'image_id', 'keypoints', 'scale', 'score'])

In [389]:
len(result)

6334

In [390]:
center = []
image_id = []
scale = []
score = []

for item in result:
    center.append(item['center'])
    image_id.append(item['image_id'])
    scale.append(item['scale'])
    score.append(item['score'])

In [391]:
result_df = pd.DataFrame({'image_id':image_id,'center':center,'scale':scale,'score':score})
result_df

,image_id,center,scale,score
0,397133,"[442.864990234375, 208.22999572753906]","[1.2966562509536743, 1.7288750410079956]",0.856345
1,397133,"[30.579999923706055, 280.69500732421875]","[0.38224998116493225, 0.5096666812896729]",0.397612
2,252219,"[361.3999938964844, 272.68499755859375]","[0.919921875, 1.2265625]",0.895720
3,252219,"[70.26000213623047, 279.7850036621094]","[1.056796908378601, 1.409062385559082]",0.895242
4,252219,"[571.77001953125, 278.6499938964844]","[1.0066875219345093, 1.3422499895095825]",0.829179
...,...,...,...,...
6329,15335,"[286.7699890136719, 99.50499725341797]","[0.6165000200271606, 0.8219999670982361]",0.508098
6330,15335,"[533.0349731445312, 99.5199966430664]","[0.601312518119812, 0.8017499446868896]",0.415471
6331,15335,"[426.94000244140625, 109.0199966430664]","[1.0413750410079956, 1.3884999752044678]",0.396422
6332,15335,"[198.88499450683594, 108.5250015258789]","[0.48118752241134644, 0.6415833234786987]",0.294983


- image_id -> score 정렬되어있음

In [392]:
# # 데이터를 평평한 형태로 변환
# flat_data = [(image_id, bbox_data) for image_id, bbox_data_list in result_df.items() for bbox_data in bbox_data_list]

# # 데이터프레임으로 변환
# result_df = pd.DataFrame(flat_data, columns=['image_id', 'bbox_data'])

# # JSON normalization을 통해 nested된 데이터를 펼침
# df_normalized = pd.json_normalize(result_df['bbox_data'])

# # 기존 'bbox_data' 열 제거 및 새로운 열로 추가
# result_df = pd.concat([result_df.drop(['bbox_data'], axis=1), df_normalized], axis=1)

# # 결과 확인
# print(result_df)

In [393]:
result_df.columns

Index(['image_id', 'center', 'scale', 'score'], dtype='object')

In [394]:
# tmp = result_df[['image_id', 'center', 'scale', 'score']]
# result_df = tmp.iloc[:,1:]
# result_df.columns

In [395]:
# bbox에서 kpts 저장되어있는 모델 중간 json 파일
# JSON 파일 경로
json_file_path = 'GT_result.json'

# JSON 파일 읽기
with open(json_file_path, 'r') as f:
    kpt_result = json.load(f)

In [396]:
kpt_result['397133'][0].keys()

dict_keys(['keypoints', 'center', 'scale', 'area', 'score', 'image_id', 'bbox_id'])

In [397]:
# 데이터프레임을 담을 리스트 생성
df_list = []

# JSON 데이터를 데이터프레임으로 변환
for image_id, bbox_list in kpt_result.items():
    for bbox_id, bbox_info in enumerate(bbox_list):
        # 필요한 정보 추출
        bbox_data = {
            "image_id": int(image_id),
            "bbox_id": bbox_id,
            "keypoints": bbox_info["keypoints"]["__ndarray__"],
            "center": bbox_info["center"]["__ndarray__"],
            "scale": bbox_info["scale"]["__ndarray__"],
            "area": bbox_info["area"],
            "score": bbox_info["score"]
        }
        df_list.append(bbox_data)

# 리스트를 데이터프레임으로 변환
kpt_df = pd.DataFrame(df_list)

In [398]:
print(kpt_df.columns)
print(len(kpt_df))

Index(['image_id', 'bbox_id', 'keypoints', 'center', 'scale', 'area', 'score'], dtype='object')
6352


In [399]:
print(kpt_df.columns)
print(result_df.columns)
print(gt_df.columns)

Index(['image_id', 'bbox_id', 'keypoints', 'center', 'scale', 'area', 'score'], dtype='object')
Index(['image_id', 'center', 'scale', 'score'], dtype='object')
Index(['image_id', 'bbox', 'width', 'height', 'size'], dtype='object')


In [400]:
result_df['center']

0         [442.864990234375, 208.22999572753906]
1       [30.579999923706055, 280.69500732421875]
2        [361.3999938964844, 272.68499755859375]
3         [70.26000213623047, 279.7850036621094]
4           [571.77001953125, 278.6499938964844]
                          ...                   
6329      [286.7699890136719, 99.50499725341797]
6330       [533.0349731445312, 99.5199966430664]
6331     [426.94000244140625, 109.0199966430664]
6332     [198.88499450683594, 108.5250015258789]
6333      [578.4849853515625, 91.56999969482422]
Name: center, Length: 6334, dtype: object

In [401]:
kpt_df['center']

0         [442.864990234375, 208.22999572753906]
1       [30.579999923706055, 280.69500732421875]
2        [361.3999938964844, 272.68499755859375]
3         [70.26000213623047, 279.7850036621094]
4           [571.77001953125, 278.6499938964844]
                          ...                   
6347                 [107.06500244140625, 259.5]
6348       [533.0349731445312, 99.5199966430664]
6349      [578.4849853515625, 91.56999969482422]
6350     [426.94000244140625, 109.0199966430664]
6351     [198.88499450683594, 108.5250015258789]
Name: center, Length: 6352, dtype: object

In [402]:
# 리스트를 풀어서 각각의 요소를 새로운 열로 만들기
kpt_df_expanded = kpt_df['center'].apply(pd.Series).rename(columns=lambda x: f'value_{x}')
result_df_expanded = result_df['center'].apply(pd.Series).rename(columns=lambda x: f'value_{x}')

In [403]:
# 기존의 key 열은 삭제
df1 = pd.concat([kpt_df, kpt_df_expanded], axis=1)
df2 = pd.concat([result_df, result_df_expanded], axis=1)

# 키를 기준으로 데이터프레임 합치기
merged_df = pd.merge(df1, df2, how='inner', on=['value_0', 'value_1'])

In [404]:
merged_df

,image_id_x,bbox_id,keypoints,center_x,scale_x,area,score_x,value_0,value_1,image_id_y,center_y,scale_y,score_y
0,397133,0,"[[430.8645324707031, 95.42893981933594, 0.9389...","[442.864990234375, 208.22999572753906]","[1.2966562509536743, 1.7288750410079956]",89670.257812,1.0,442.864990,208.229996,397133,"[442.864990234375, 208.22999572753906]","[1.2966562509536743, 1.7288750410079956]",0.856345
1,397133,1,"[[52.14830780029297, 284.97412109375, 0.023037...","[30.579999923706055, 280.69500732421875]","[0.38224998116493225, 0.5096666812896729]",7792.803223,1.0,30.580000,280.695007,397133,"[30.579999923706055, 280.69500732421875]","[0.38224998116493225, 0.5096666812896729]",0.397612
2,252219,0,"[[354.296630859375, 198.06585693359375, 0.9260...","[361.3999938964844, 272.68499755859375]","[0.919921875, 1.2265625]",45133.667969,1.0,361.399994,272.684998,252219,"[361.3999938964844, 272.68499755859375]","[0.919921875, 1.2265625]",0.895720
3,252219,1,"[[100.3197021484375, 189.2662811279297, 0.9693...","[70.26000213623047, 279.7850036621094]","[1.056796908378601, 1.409062385559082]",59563.707031,1.0,70.260002,279.785004,252219,"[70.26000213623047, 279.7850036621094]","[1.056796908378601, 1.409062385559082]",0.895242
4,252219,2,"[[537.075927734375, 193.65164184570312, 0.9272...","[571.77001953125, 278.6499938964844]","[1.0066875219345093, 1.3422499895095825]",54049.058594,1.0,571.770020,278.649994,252219,"[571.77001953125, 278.6499938964844]","[1.0066875219345093, 1.3422499895095825]",0.829179
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6329,15335,3,"[[83.04756164550781, 175.28688049316406, 0.925...","[107.06500244140625, 259.5]","[1.7588437795639038, 2.3451249599456787]",164988.343750,1.0,107.065002,259.500000,15335,"[107.06500244140625, 259.5]","[1.7588437795639038, 2.3451249599456787]",0.714482
6330,15335,4,"[[508.6524963378906, 30.947364807128906, 0.216...","[533.0349731445312, 99.5199966430664]","[0.601312518119812, 0.8017499446868896]",19284.091797,1.0,533.034973,99.519997,15335,"[533.0349731445312, 99.5199966430664]","[0.601312518119812, 0.8017499446868896]",0.415471
6331,15335,5,"[[561.7669067382812, 92.91932678222656, 0.1978...","[578.4849853515625, 91.56999969482422]","[0.6596249938011169, 0.8794999718666077]",23205.607422,1.0,578.484985,91.570000,15335,"[578.4849853515625, 91.56999969482422]","[0.6596249938011169, 0.8794999718666077]",0.282995
6332,15335,6,"[[408.19903564453125, 93.58609008789062, 0.499...","[426.94000244140625, 109.0199966430664]","[1.0413750410079956, 1.3884999752044678]",57837.964844,1.0,426.940002,109.019997,15335,"[426.94000244140625, 109.0199966430664]","[1.0413750410079956, 1.3884999752044678]",0.396422


In [405]:
print(result_df.columns)
print(kpt_df.columns)

Index(['image_id', 'center', 'scale', 'score'], dtype='object')
Index(['image_id', 'bbox_id', 'keypoints', 'center', 'scale', 'area', 'score'], dtype='object')


In [406]:
merged_df.iloc[0]

image_id_x                                               397133
bbox_id                                                       0
keypoints     [[430.8645324707031, 95.42893981933594, 0.9389...
center_x                 [442.864990234375, 208.22999572753906]
scale_x                [1.2966562509536743, 1.7288750410079956]
area                                               89670.257812
score_x                                                     1.0
value_0                                               442.86499
value_1                                              208.229996
image_id_y                                               397133
center_y                 [442.864990234375, 208.22999572753906]
scale_y                [1.2966562509536743, 1.7288750410079956]
score_y                                                0.856345
Name: 0, dtype: object

살릴거
- image_id_x
- bbox_id
- center_x
- scale_x
- score_y : keypoints score

In [407]:
merged_df = merged_df[['image_id_x', 'bbox_id','center_x','scale_x','score_y']]

In [408]:
# 두 개의 열을 기준으로 정렬 (예: Column1을 오름차순으로 정렬하고, 그 후에 Column2를 오름차순으로 정렬)
merged_df = merged_df.sort_values(by=['image_id_x', 'score_y'], ascending=[True, True]).reset_index(drop=True)

print(merged_df)

      image_id_x  bbox_id                                  center_x  \
0            139        0  [438.82501220703125, 226.11500549316406]   
1            785        0   [389.6400146484375, 217.57000732421875]   
2            872        0    [290.7349853515625, 328.8450012207031]   
3            885        2    [616.864990234375, 139.02000427246094]   
4            885        0    [346.8550109863281, 293.6000061035156]   
...          ...      ...                                       ...   
6329      580294        0   [525.0700073242188, 220.07000732421875]   
6330      581062        0   [162.58999633789062, 107.5250015258789]   
6331      581206        0  [234.26499938964844, 109.66000366210938]   
6332      581317        0  [511.06500244140625, 204.46499633789062]   
6333      581357        0  [372.20001220703125, 216.56500244140625]   

                                       scale_x   score_y  
0     [0.6422343850135803, 0.8563125133514404]  0.763357  
1     [1.6203750371932983, 2.

In [409]:
gt_df = gt_df.sort_values(by='image_id', ascending=True).reset_index(drop=True)
gt_df

,image_id,bbox,width,height,size
0,139,"[384.43, 172.21, 15.12, 35.74]",15.12,35.74,540.3888
1,139,"[412.8, 157.61, 53.05, 138.01]",53.05,138.01,7321.4305
2,785,"[280.79, 44.73, 218.7, 346.68]",218.70,346.68,75818.9160
3,872,"[145.26, 100.67, 291.95, 457.35]",291.95,457.35,133523.3325
4,872,"[163.73, 126.42, 265.69, 480.4]",265.69,480.40,127637.4760
...,...,...,...,...,...
10999,581357,"[578.74, 389.17, 12.86, 25.39]",12.86,25.39,326.5154
11000,581357,"[262.68, 93.29, 220.04, 247.55]",220.04,247.55,54470.9020
11001,581357,"[249.22, 419.65, 13.2, 24.53]",13.20,24.53,323.7960
11002,581357,"[428.92, 400.24, 22.69, 45.38]",22.69,45.38,1029.6722


In [410]:
merged_df.to_csv("merged_df.csv", index=False)
gt_df.to_csv("gt_df.csv", index=False)